<a href="https://colab.research.google.com/github/balawillgetyou/ax/blob/master/AmexAV20191004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import zipfile

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1) #to prevent cell display truncation

import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
zip_ref = zipfile.ZipFile("/content/gdrive/My Drive/Colab Notebooks/train_AUpWtIz.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()
filesInFolder = !ls "/tmp"
print(filesInFolder)
files_csv = [f for f in filesInFolder if (f[-3:] == 'csv')]
print(files_csv)

['campaign_data.csv\t       drivefs_ipc.0\t    tmp55fk360r', 'coupon_item_mapping.csv        drivefs_ipc.0_shell  train.csv', 'customer_demographics.csv      item_data.csv', 'customer_transaction_data.csv  __MACOSX']
['coupon_item_mapping.csv        drivefs_ipc.0_shell  train.csv', 'customer_demographics.csv      item_data.csv']


In [5]:
temp_1 = open("/tmp/train.csv", 'r', encoding='latin-1') 
train = pd.read_csv(temp_1)
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [6]:
temp_1 = open("/tmp/item_data.csv", 'r', encoding='latin-1') 
item_data = pd.read_csv(temp_1)
item_data.head()

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [7]:
temp_1 = open("/tmp/customer_transaction_data.csv", 'r', encoding='latin-1') 
customer_transaction_data = pd.read_csv(temp_1)
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [8]:
temp_1 = open("/tmp/customer_demographics.csv", 'r', encoding='latin-1') 
customer_demographics = pd.read_csv(temp_1)
print(customer_demographics.dtypes)
customer_demographics.head()


customer_id       int64 
age_range         object
marital_status    object
rented            int64 
family_size       object
no_of_children    object
income_bracket    int64 
dtype: object


,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5


In [10]:
Columns = customer_demographics.columns.to_list()
customer_demographics[Columns] = customer_demographics[Columns].apply(pd.Categorical)
customer_demographics.dtypes

customer_id       category
age_range         category
marital_status    category
rented            category
family_size       category
no_of_children    category
income_bracket    category
dtype: object

In [9]:
temp_1 = open("/tmp/coupon_item_mapping.csv", 'r', encoding='latin-1') 
coupon_item_mapping = pd.read_csv(temp_1)
coupon_item_mapping.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [11]:
temp_1 = open("/tmp/campaign_data.csv", 'r', encoding='latin-1') 
campaign_data = pd.read_csv(temp_1)
campaign_data.head()

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13
3,23,Y,08/10/13,15/11/13
4,21,Y,16/09/13,18/10/13


In [13]:
from google.colab import files
uploaded = files.upload()

Saving test_QyjYwdj.csv to test_QyjYwdj.csv


In [14]:
temp_1 = open("test_QyjYwdj.csv", 'r', encoding='latin-1') 
test = pd.read_csv(temp_1)
test.head()

,id,campaign_id,coupon_id,customer_id
0,3,22,869,967
1,4,20,389,1566
2,5,22,981,510
3,8,25,1069,361
4,10,17,498,811


In [16]:
train_coupon_item_customer = pd.merge(train, coupon_item_mapping, 'left', on='coupon_id')
train_coupon_item_customer = pd.merge(train_coupon_item_customer, item_data, 'left', on='item_id')
train_coupon_item_customer = pd.merge(train_coupon_item_customer, customer_demographics, 'left', on='customer_id')
train_coupon_item_customer.shape

(6420694, 15)

In [0]:
commonCoupons = list(set(test.coupon_id).intersection(train_coupon_item_customer.coupon_id))
commonCustomers = list(set(test.customer_id).intersection(train_coupon_item_customer.customer_id))

In [18]:
train_coupon_item_customer = train_coupon_item_customer[train_coupon_item_customer['coupon_id'].isin(commonCoupons)]
train_coupon_item_customer = train_coupon_item_customer[train_coupon_item_customer['customer_id'].isin(commonCustomers)]
train_coupon_item_customer.shape

(402537, 15)

In [20]:
train_coupon_item_customer = train_coupon_item_customer[['redemption_status', 'coupon_id', 'customer_id', 'item_id',
 'brand', 'brand_type', 'category', 'age_range', 'rented', 'family_size', 'income_bracket']]
train_coupon_item_customer = train_coupon_item_customer.apply(pd.Categorical) 
train_coupon_item_customer.dropna(inplace = True)
print(train_coupon_item_customer.isnull().sum())
#train_coupon_item_customer.shape
print(train_coupon_item_customer.dtypes)

redemption_status    0
coupon_id            0
customer_id          0
item_id              0
brand                0
brand_type           0
category             0
age_range            0
rented               0
family_size          0
income_bracket       0
dtype: int64
redemption_status    category
coupon_id            category
customer_id          category
item_id              category
brand                category
brand_type           category
category             category
age_range            category
rented               category
family_size          category
income_bracket       category
dtype: object


In [0]:
train_coupon_item_customerX = train_coupon_item_customer.drop('redemption_status', axis=1)
train_coupon_item_customerY = train_coupon_item_customer.redemption_status

le_age_range = LabelEncoder()
le_age_range.fit(train_coupon_item_customerX['age_range'])
train_coupon_item_customerX['age_range'] = le_age_range.transform(train_coupon_item_customerX['age_range'])

le_family_size = LabelEncoder()
le_family_size.fit(train_coupon_item_customerX['family_size'])
train_coupon_item_customerX['family_size'] = le_family_size.transform(train_coupon_item_customerX['family_size'])

le_brand_type = LabelEncoder()
le_brand_type.fit(train_coupon_item_customerX['brand_type'])
train_coupon_item_customerX['brand_type'] = le_brand_type.transform(train_coupon_item_customerX['brand_type'])

le_category = LabelEncoder()
le_category.fit(train_coupon_item_customerX['category'])
train_coupon_item_customerX['category'] = le_category.transform(train_coupon_item_customerX['category'])

# implementing train-test-split
X_train, X_test, y_train, y_test = train_test_split(train_coupon_item_customerX, train_coupon_item_customerY, test_size=0.05, random_state=66)

In [0]:
# random forest model creation
rfc = RandomForestClassifier(n_estimators=100, oob_score=True)
rfc.fit(X_train,y_train)
# predictions
rfc_predict = rfc.predict(X_test)

In [25]:
rfc.oob_score_

0.9889968125520945

In [0]:
rfc_cv_score = cross_val_score(rfc, train_coupon_item_customerX, train_coupon_item_customerY, cv=10, scoring='roc_auc')

In [28]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[11987    57]
 [   69   453]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     12044
           1       0.89      0.87      0.88       522

    accuracy                           0.99     12566
   macro avg       0.94      0.93      0.94     12566
weighted avg       0.99      0.99      0.99     12566



=== All AUC Scores ===
[0.78417041 0.71622108 0.81627552 0.76934176 0.85395019 0.71774211
 0.70392764 0.69190643 0.72968297 0.65017111]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.7433389235335841


In [33]:
print(train.shape)
print(customer_demographics.shape)

(78369, 5)
(760, 7)


In [32]:
train_customer = pd.merge(train, customer_demographics, 'left', on='customer_id')
train_customer.shape

(78369, 11)

In [34]:
train_customer.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'age_range', 'marital_status', 'rented', 'family_size',
       'no_of_children', 'income_bracket'],
      dtype='object')

In [36]:
commonCustomers = list(set(test.customer_id).intersection(train_customer.customer_id))
train_customer = train_customer[train_customer['customer_id'].isin(commonCustomers)]
print(train_customer.shape)
train_customer = train_customer[['redemption_status', 'coupon_id', 'customer_id', 'age_range', 'rented', 'family_size', 'income_bracket']]
train_customer = train_customer.apply(pd.Categorical) 
train_customer.dropna(inplace = True)
print(train_customer.isnull().sum())
#train_coupon_item_customer.shape
print(train_customer.dtypes)

(66054, 11)
redemption_status    0
coupon_id            0
customer_id          0
age_range            0
rented               0
family_size          0
income_bracket       0
dtype: int64
redemption_status    category
coupon_id            category
customer_id          category
age_range            category
rented               category
family_size          category
income_bracket       category
dtype: object


In [0]:
train_customerX = train_customer.drop('redemption_status', axis=1)
train_customerY = train_customer.redemption_status

le_age_range = LabelEncoder()
le_age_range.fit(train_customerX['age_range'])
train_customerX['age_range'] = le_age_range.transform(train_customerX['age_range'])

le_family_size = LabelEncoder()
le_family_size.fit(train_customerX['family_size'])
train_customerX['family_size'] = le_family_size.transform(train_customerX['family_size'])

# implementing train-test-split
X_train, X_test, y_train, y_test = train_test_split(train_customerX, train_customerY, test_size=0.05, random_state=66)

In [38]:
# random forest model creation
rfc = RandomForestClassifier(n_estimators=100, oob_score=True)
rfc.fit(X_train,y_train)
# predictions
rfc_predict = rfc.predict(X_test)
rfc.oob_score_

0.9824435133537259

In [39]:
rfc_cv_score = cross_val_score(rfc, train_coupon_item_customerX, train_coupon_item_customerY, cv=10, scoring='roc_auc')

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[2003    9]
 [  30    6]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2012
           1       0.40      0.17      0.24        36

    accuracy                           0.98      2048
   macro avg       0.69      0.58      0.61      2048
weighted avg       0.97      0.98      0.98      2048



=== All AUC Scores ===
[0.72353866 0.71770195 0.83170897 0.7874062  0.87724077 0.70260045
 0.67510525 0.73408369 0.7424312  0.71405913]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.750587624274389


In [44]:
print(test.shape)
print(test.isnull().sum())
testMod = pd.merge(test, customer_demographics, 'left', on='customer_id')
print(testMod.shape)
print(testMod.columns)
print(testMod.isnull().sum())

(50226, 4)
id             0
campaign_id    0
coupon_id      0
customer_id    0
dtype: int64
(50226, 10)
Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'age_range',
       'marital_status', 'rented', 'family_size', 'no_of_children',
       'income_bracket'],
      dtype='object')
id                0    
campaign_id       0    
coupon_id         0    
customer_id       0    
age_range         19287
marital_status    32144
rented            19287
family_size       19287
no_of_children    40064
income_bracket    19287
dtype: int64


In [45]:
testMod = testMod[['coupon_id', 'customer_id', 'age_range', 'rented', 'family_size', 'income_bracket']]
testMod = testMod.apply(pd.Categorical) 
#train_customer.dropna(inplace = True)
print(testMod.isnull().sum())
#train_coupon_item_customer.shape
print(testMod.dtypes)

coupon_id         0    
customer_id       0    
age_range         19287
rented            19287
family_size       19287
income_bracket    19287
dtype: int64
coupon_id         category
customer_id       category
age_range         category
rented            category
family_size       category
income_bracket    category
dtype: object


In [54]:
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


________________________________________________________________________________________________________________________________________________________________

In [0]:
customer_transaction_data['date'] = pd.to_datetime(customer_transaction_data.date)
customer_transaction_dataMod = customer_transaction_data
customer_transaction_dataMod = customer_transaction_dataMod.set_index('date')
customer_transaction_dataMod['usedOtherDisc'] = np.where(customer_transaction_dataMod['other_discount']<0, 'yes', 'no')
customer_transaction_dataMod['usedCouponDisc'] = np.where(customer_transaction_dataMod['coupon_discount']<0, 'yes', 'no')

In [0]:
customer_transaction_dataMod.head()